In [6]:
!pip install torchvision

In [1]:
import torchvision.transforms as transforms

mnist_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (1.0,))
])


In [3]:
from torchvision.datasets import MNIST
import requests

download_root = '../chap02/data/MNIST_DATASET'

train_dataset = MNIST(download_root, transform = mnist_transform, train= True, download=True)

valid_dataset = MNIST(download_root, transform = mnist_transform, train= False, download=True)

test_dataset = MNIST(download_root, transform = mnist_transform, train= False, download=True)

In [4]:
import torch.nn as nn

train_dataset

Dataset MNIST
    Number of datapoints: 60000
    Root location: ../chap02/data/MNIST_DATASET
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5,), std=(1.0,))
           )

In [7]:
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [8]:
device

device(type='cuda', index=0)

In [10]:
!pip install matplotlib

  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
     ---------------------------------------- 0.0/165.9 kB ? eta -:--:--
     ------ ------------------------------ 30.7/165.9 kB 640.0 kB/s eta 0:00:01
     -------------------------------------  163.8/165.9 kB 2.4 MB/s eta 0:00:01
     -------------------------------------- 165.9/165.9 kB 2.0 MB/s eta 0:00:00
  Using cached pyparsing-3.1.2-py3-none-any.whl.metadata (5.1 kB)
   ---------------------------------------- 0.0/8.0 MB ? eta -:--:--
   -- ------------------------------------- 0.6/8.0 MB 12.7 MB/s eta 0:00:01
   ------ --------------------------------- 1.4/8.0 MB 14.4 MB/s eta 0:00:01
   --------- ------------------------------ 1.9/8.0 MB 13.2 MB/s eta 0:00:01
   ------------- -------------------------- 2.7/8.0 MB 14.5 MB/s eta 0:00:01
   ----------------- ---------------------- 3.5/8.0 MB 16.1 MB/s eta 0:00:01
   --------------------- ------------------ 4.2/8.0 MB 16.7 MB/s eta 0:00:01
   --------------------- -

In [12]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

In [13]:
train_dataset = torchvision.datasets.FashionMNIST("../chap05/data", download=True, transform=transforms.Compose([transforms.ToTensor()]))
test_dataset = torchvision.datasets.FashionMNIST("../chap05/data", download=True, train=False, transform=transforms.Compose([transforms.ToTensor()]))

100.0%


Extracting ../chap05/data\FashionMNIST\raw\train-images-idx3-ubyte.gz to ../chap05/data\FashionMNIST\raw



100.0%


Extracting ../chap05/data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to ../chap05/data\FashionMNIST\raw



100.0%


Extracting ../chap05/data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to ../chap05/data\FashionMNIST\raw



100.0%

Extracting ../chap05/data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to ../chap05/data\FashionMNIST\raw



In [14]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=100)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100)


In [15]:
labels_map = {0 : 'T-Shirt', 1 : 'Trouser', 2 : 'Pullover', 3 : 'Dress', 4 : 'Coat', 5 : 'Sandal', 6 : 'Shirt',
              7 : 'Sneaker', 8 : 'Bag', 9 : 'Ankle Boot'}

In [19]:
class FashionDNN(nn.Module):
    def __init__(self):
        super(FashionDNN, self).__init__()
        self.fc1 = nn.Linear(in_features=784, out_features=256)
        self.drop = nn.Dropout(0.25)
        self.fc2 = nn.Linear(in_features=256, out_features=128)
        self.fc3 = nn.Linear(in_features=128, out_features=10)

    def forward(self, input_data):
        out = input_data.view(-1, 784)
        out = F.relu(self.fc1(out))
        out = self.drop(out)
        out = F.relu(self.fc2(out))
        out = self.fc3(out)
        return out

In [20]:
learning_rate = 0.001;
model = FashionDNN();
model.to(device)

criterion = nn.CrossEntropyLoss();
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate);
print(model)

FashionDNN(
  (fc1): Linear(in_features=784, out_features=256, bias=True)
  (drop): Dropout(p=0.25, inplace=False)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=10, bias=True)
)


In [21]:
num_epochs = 5
count = 0
loss_list = []
iteration_list = []
accuracy_list = []

predictions_list = []
labels_list = []

for epoch in range(num_epochs):
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
    
        train = images.view(100, 1, 28, 28)
        labels = labels
        
        outputs = model(train)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        count += 1
    
        if not (count % 50):    
            total = 0
            correct = 0        
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                labels_list.append(labels)            
                test = images.view(100, 1, 28, 28)        
                outputs = model(test)            
                predictions = torch.max(outputs, 1)[1].to(device)
                predictions_list.append(predictions)
                correct += (predictions == labels).sum()            
                total += len(labels)
            
            accuracy = correct * 100 / total
            loss_list.append(loss.data)
            iteration_list.append(count)
            accuracy_list.append(accuracy)
        
        if not (count % 500):
            print("Iteration: {}, Loss: {}, Accuracy: {}%".format(count, loss.data, accuracy))

Iteration: 500, Loss: 0.6281203627586365, Accuracy: 83.1199951171875%
Iteration: 1000, Loss: 0.4296606183052063, Accuracy: 84.80999755859375%
Iteration: 1500, Loss: 0.36627623438835144, Accuracy: 83.87999725341797%
Iteration: 2000, Loss: 0.41814789175987244, Accuracy: 85.18999481201172%
Iteration: 2500, Loss: 0.2955018877983093, Accuracy: 86.30999755859375%
Iteration: 3000, Loss: 0.3149814009666443, Accuracy: 86.27999877929688%
